In [20]:
import polars as pl
import re

# Load the Bronze layer for AAPL
df = pl.read_parquet("/Users/architmanek/Desktop/DataEngineering/momentum_pipeline/data/silver/AAPL.parquet")

# Preview the shape and schema
# print(df.shape)
# print(df.schema)

# Look at the first few rows
print(df.columns)
df.head(10)


['Date', 'Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL', 'Volume_AAPL', 'daily_log_return']


Date,Close_AAPL,High_AAPL,Low_AAPL,Open_AAPL,Volume_AAPL,daily_log_return
datetime[ns],f64,f64,f64,f64,i64,f64
2015-01-02 00:00:00,24.288584,24.757338,23.848709,24.74623,212818400,null
2015-01-05 00:00:00,23.604328,24.137509,23.417716,24.057531,257142000,-0.028576
2015-01-06 00:00:00,23.606558,23.866483,23.244438,23.668762,263188400,0.000094
2015-01-07 00:00:00,23.937571,24.037541,23.704304,23.815383,160423600,0.013925
2015-01-08 00:00:00,24.857306,24.915067,24.14862,24.266365,237458000,0.037702
2015-01-09 00:00:00,24.883966,25.159442,24.484081,25.03059,214798000,0.001072
2015-01-12 00:00:00,24.270809,25.021704,24.170839,25.01504,198603200,-0.024949
2015-01-13 00:00:00,24.486301,25.059471,24.195274,24.755113,268367600,0.008839
2015-01-14 00:00:00,24.392996,24.546284,24.104189,24.224155,195826400,-0.003818


In [19]:
from dagster import asset
import yfinance as yf
import polars as pl
from pathlib import Path
import pandas as pd

TICKER = "AAPL"
START_DATE = "2015-01-01"
END_DATE = "2025-01-01"

df = yf.download(TICKER, start=START_DATE, end=END_DATE)
if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(filter(None, col)).strip() for col in df.columns]
df.reset_index(inplace=True)
print(df.columns)
pl_df = pl.from_pandas(df)

print(pl_df.head(10))

[*********************100%***********************]  1 of 1 completed

Index(['Date', 'Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL',
       'Volume_AAPL'],
      dtype='object')
shape: (10, 6)
┌─────────────────────┬────────────┬───────────┬───────────┬───────────┬─────────────┐
│ Date                ┆ Close_AAPL ┆ High_AAPL ┆ Low_AAPL  ┆ Open_AAPL ┆ Volume_AAPL │
│ ---                 ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---         │
│ datetime[ns]        ┆ f64        ┆ f64       ┆ f64       ┆ f64       ┆ i64         │
╞═════════════════════╪════════════╪═══════════╪═══════════╪═══════════╪═════════════╡
│ 2015-01-02 00:00:00 ┆ 24.288578  ┆ 24.757332 ┆ 23.848704 ┆ 24.746224 ┆ 212818400   │
│ 2015-01-05 00:00:00 ┆ 23.604336  ┆ 24.137516 ┆ 23.417723 ┆ 24.057539 ┆ 257142000   │
│ 2015-01-06 00:00:00 ┆ 23.606562  ┆ 23.866487 ┆ 23.244442 ┆ 23.668766 ┆ 263188400   │
│ 2015-01-07 00:00:00 ┆ 23.937572  ┆ 24.037543 ┆ 23.704305 ┆ 23.815385 ┆ 160423600   │
│ 2015-01-08 00:00:00 ┆ 24.857304  ┆ 24.915065 ┆ 24.148618 ┆ 24.266363 ┆ 237458000   │
│ 2